<a href="https://colab.research.google.com/github/laurence-lin/Kaggle_competition/blob/master/FraudDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
import lightgbm as lgb
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler


import os
import gc
from time import time


In [2]:
from google.colab import files, drive

drive.mount('/content/gdrive')
%cd /content/gdrive/"My Drive"
!unzip ieee-fraud-detection.zip

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive
Archive:  ieee-fraud-detection.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace test_identity.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [24]:
train_id = pd.read_csv('train_identity.csv')
train_action = pd.read_csv('train_transaction.csv')
test_id = pd.read_csv('test_identity.csv')
test_action = pd.read_csv('test_transaction.csv')

# Merge datasets: not all transaction record has corresponding Identity informationb
train = train_action.merge(train_id, on = 'TransactionID', how = 'left')
test = test_action.merge(test_id, on = 'TransactionID', how = 'left')
del train_action, test_action, train_id, test_id
gc.collect()

364

*We* have 590000 training samples, and over 400 feature in total. 
We may need feature selection for FE.

In [0]:
# EDA

# Too many feature, we need feature selection afterward
print(train.isnull().sum()[train.isnull().sum() > int(train.shape[0]*0.7)].sort_values(ascending = False))
drop_feature = list(train.isnull().sum()[train.isnull().sum() > int(train.shape[0]*0.7)].index)

print(drop_feature)
print(train[drop_feature])

In [0]:
# Feature engineering

def preprocessing(df):
  '''
  Feature engineering for dataframe.
  '''
 
  # Feature Generation

  # Feature selection
  # There are many features, so we may drop the ones that have 70% nan values
  drop_nan = list(df.isnull().sum()[df.isnull().sum() > int(df.shape[0]*0.7)].index)
  df.drop(drop_nan, axis = 1, inplace = True)

  # Data cleaning
  # fill missing

  return df

In [25]:

le = LabelEncoder()

train_ = train.copy()
train_ = preprocessing(train_)

obj_feature = list(train_.select_dtypes(include = 'object').columns)
for feat in obj_feature:
  train_[feat] = le.fit_transform(train_[feat].astype('str'))


num_feature = list(train_.select_dtypes(exclude = 'object'))
for f in num_feature:
    train_[f] = train_[f].fillna(train_[f].mean(), inplace = False)

y = train_['isFraud']
x = train_.drop('isFraud', axis = 1, inplace = False)
x_data, x_valid, y_data, y_valid = train_test_split(x, y, test_size = 0.2)

logit = LogisticRegression()
logit.fit(x_data, y_data)
print('ROC accuracy: ', roc_auc_score(y_valid, logit.predict(x_valid)))

del train_
gc.collect()

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logit score:  0.9646255969113016


0